<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Iowa Liquor Sales 

---

Project 3, unlike project 2 which combined skills from weeks 1 and 2, will now be review of week 2. Projects going forward will fit this format: you learn material one week and review it through a project the following week.

That being said, this is as you might expect more challenging. You will be using the pandas data transformation functionality (melt, pivot, groupby, apply) that was not a factor in Project 2.

---

#### Context of the data

The state of Iowa provides many data sets on their website, including [this dataset](https://www.dropbox.com/sh/pf5n5sgfgiri3i8/AACkaMeL_i_WgZ00rpxOOcysa?dl=0) which contains transactions for all stores that have a class E liquor license. You can choose one of the following two scenarios.

The data can also be found [directly on their website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy), which allows you to explore it graphically and download it (though it doesn't work very well).

NOTE: Some of you may have computer issues with the full dataset. In this case, feel free to use [this 10% dataset version of Iowa liquor sales](https://drive.google.com/file/d/0Bx2SHQGVqWaseDB4QU9ZSVFDY2M/view?usp=sharing). You may want to use it anyway to test and prototype your code since it will be faster, before running it on the full dataset.

---

#### Package imports

In [1]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import csv
import datetime
import seaborn as sns
import matplotlib.pyplot as plt


# this line tells jupyter notebook to put the plots in the notebook rather than saving them to file.
%matplotlib inline

# this line makes plots prettier on mac retina screens. If you don't have one it shouldn't do anything.
%config InlineBackend.figure_format = 'retina'

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Load the data from file

---

In [2]:
iowa = pd.read_csv('/Users/VanessaG/desktop/DSI-SF-2-vnessified/datasets/iowa_liquor/Iowa_Liquor_sales_sample_10pct.csv')

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Do an initial overview of the data

---

At the very least describe the columns/variables and the datatypes. 

In [3]:
iowa.head()

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,11/04/2015,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,$4.50,$6.75,12,$81.00,9.0,2.38
1,03/02/2016,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,$13.75,$20.63,2,$41.26,1.5,0.40
2,02/11/2016,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,$12.59,$18.89,24,$453.36,24.0,6.34
3,02/03/2016,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,$9.50,$14.25,6,$85.50,10.5,2.77
4,08/18/2015,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,$7.20,$10.80,12,$129.60,21.0,5.55


In [4]:
#whoa how large is the original
iowa.shape

(270955, 18)

In [5]:
iowa.columns

Index([u'Date', u'Store Number', u'City', u'Zip Code', u'County Number',
       u'County', u'Category', u'Category Name', u'Vendor Number',
       u'Item Number', u'Item Description', u'Bottle Volume (ml)',
       u'State Bottle Cost', u'State Bottle Retail', u'Bottles Sold',
       u'Sale (Dollars)', u'Volume Sold (Liters)', u'Volume Sold (Gallons)'],
      dtype='object')

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Clean the dataset

---

### 3.1 Identify columns that you will need to convert and clean. Where and how is the data corrupted?

Don't worry about converting the date column to a pandas/numpy "datetime" datatype, unless you want to (not required for these problems and is a challenging thing to work with in its own right.)

In [6]:
iowa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270955 entries, 0 to 270954
Data columns (total 18 columns):
Date                     270955 non-null object
Store Number             270955 non-null int64
City                     270955 non-null object
Zip Code                 270955 non-null object
County Number            269878 non-null float64
County                   269878 non-null object
Category                 270887 non-null float64
Category Name            270323 non-null object
Vendor Number            270955 non-null int64
Item Number              270955 non-null int64
Item Description         270955 non-null object
Bottle Volume (ml)       270955 non-null int64
State Bottle Cost        270955 non-null object
State Bottle Retail      270955 non-null object
Bottles Sold             270955 non-null int64
Sale (Dollars)           270955 non-null object
Volume Sold (Liters)     270955 non-null float64
Volume Sold (Gallons)    270955 non-null float64
dtypes: float64(4), int64(

**To clean:**

* Date: convert object to datetime
* County Number: missing values
* County: missing values
* Category: missing values
* Category Name: missing values
* State Bottle Cost: remove dollar sign, convert object to float
* State Bottle Retail: remove dollar sign, convert object to float
* Sale (Dollars): remove dollar sign, convert object to float

I know zip code is a categorical variable but is there some benefit to converting it to a continuous var

### 3.2 Clean the columns

In [7]:
iowa['State Bottle Cost'].head()

0     $4.50
1    $13.75
2    $12.59
3     $9.50
4     $7.20
Name: State Bottle Cost, dtype: object

In [8]:
iowa['State Bottle Retail'].head()

0     $6.75
1    $20.63
2    $18.89
3    $14.25
4    $10.80
Name: State Bottle Retail, dtype: object

In [9]:
iowa['Sale (Dollars)'].head()

0     $81.00
1     $41.26
2    $453.36
3     $85.50
4    $129.60
Name: Sale (Dollars), dtype: object

In [10]:
iowa.ix[:,['State Bottle Cost','State Bottle Retail','Sale (Dollars)']] = iowa.ix[:,['State Bottle Cost','State Bottle Retail','Sale (Dollars)']].replace('[\$,]', '', regex=True).astype(float)

In [11]:
# iowa['State Bottle Cost'].dtype
# iowa['State Bottle Retail'].dtype
iowa['Sale (Dollars)'].dtype

dtype('float64')

In [12]:
iowa.isnull().sum()

Date                        0
Store Number                0
City                        0
Zip Code                    0
County Number            1077
County                   1077
Category                   68
Category Name             632
Vendor Number               0
Item Number                 0
Item Description            0
Bottle Volume (ml)          0
State Bottle Cost           0
State Bottle Retail         0
Bottles Sold                0
Sale (Dollars)              0
Volume Sold (Liters)        0
Volume Sold (Gallons)       0
dtype: int64

In [13]:
#dropping since the number of missing is relatively low
iowa = iowa.dropna()

In [14]:
iowa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269258 entries, 0 to 270954
Data columns (total 18 columns):
Date                     269258 non-null object
Store Number             269258 non-null int64
City                     269258 non-null object
Zip Code                 269258 non-null object
County Number            269258 non-null float64
County                   269258 non-null object
Category                 269258 non-null float64
Category Name            269258 non-null object
Vendor Number            269258 non-null int64
Item Number              269258 non-null int64
Item Description         269258 non-null object
Bottle Volume (ml)       269258 non-null int64
State Bottle Cost        269258 non-null float64
State Bottle Retail      269258 non-null float64
Bottles Sold             269258 non-null int64
Sale (Dollars)           269258 non-null float64
Volume Sold (Liters)     269258 non-null float64
Volume Sold (Gallons)    269258 non-null float64
dtypes: float64(7), int

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 3.3 Perform more extensive cleaning of the dataset

Cleaning of data can mean a lot more than just fixing strings and numbers in columns. There are often logical errors with data, useless or nonsensical categories, redundancy of information, outliers, and many more problems.

This dataset has problems beyond just fixing the types of columns. Though resolving them may not be required for EDA and analysis, if you want experience with "deeper" cleaning of data this is a great dataset to start practicing with.

Keep in mind that some types of "data cleaning" is subjective: it's not always a cut-and-dry conversion of type or removal of null values. Subjectivity when dealing with data is just a fact of life for a data scientist. This isn't a kind of programming where things are just right or wrong.

In [15]:
#rename columns
iowa.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
iowa.rename(columns=lambda x: x.replace('(', ''), inplace=True)
iowa.rename(columns=lambda x: x.replace(')', ''), inplace=True)
iowa.rename(columns=lambda x: x.lower(), inplace=True)

In [16]:
iowa.columns

Index([u'date', u'store_number', u'city', u'zip_code', u'county_number',
       u'county', u'category', u'category_name', u'vendor_number',
       u'item_number', u'item_description', u'bottle_volume_ml',
       u'state_bottle_cost', u'state_bottle_retail', u'bottles_sold',
       u'sale_dollars', u'volume_sold_liters', u'volume_sold_gallons'],
      dtype='object')

In [17]:
# this was very slow operation... until I added format
iowa['date'] = pd.to_datetime(iowa['date'], format='%m/%d/%Y')

In [18]:
iowa.date.describe()

count                  269258
unique                    274
top       2015-12-01 00:00:00
freq                     1554
first     2015-01-05 00:00:00
last      2016-03-31 00:00:00
Name: date, dtype: object

In [19]:
#outliers here maybe?
iowa.store_number.value_counts()

2633    2465
4829    2285
2512    1755
2572    1643
2190    1615
2515    1560
2501    1548
2500    1519
2670    1501
2587    1479
2648    1441
2620    1433
2603    1404
2590    1353
2629    1331
2666    1312
2614    1304
2565    1303
2647    1294
2616    1286
2619    1258
2561    1248
4129    1247
2623    1234
2552    1196
2625    1180
2506    1173
3773    1172
4167    1136
3952    1135
        ... 
9013       6
5191       6
4497       6
4610       6
2961       6
5202       6
4338       6
4059       5
5139       5
4121       5
5033       5
5189       5
4990       5
4737       4
4954       4
4335       4
4855       4
4834       3
4939       3
5195       3
5193       3
5201       3
5192       2
4778       2
4567       2
4776       2
5130       2
5053       1
9018       1
9023       1
Name: store_number, dtype: int64

In [20]:
iowa.city.value_counts()

DES MOINES         23618
CEDAR RAPIDS       18736
DAVENPORT          11469
WATERLOO            8376
COUNCIL BLUFFS      8037
IOWA CITY           7938
SIOUX CITY          7888
AMES                7534
WEST DES MOINES     7148
DUBUQUE             6854
CEDAR FALLS         5719
ANKENY              4823
MASON CITY          4119
BETTENDORF          3699
CORALVILLE          3446
MUSCATINE           3389
BURLINGTON          3137
CLINTON             3077
FORT DODGE          2972
WINDSOR HEIGHTS     2797
MARSHALLTOWN        2682
NEWTON              2538
STORM LAKE          2522
MARION              2485
URBANDALE           2424
OTTUMWA             2290
JOHNSTON            2137
ALTOONA             2103
CLEAR LAKE          2080
SPENCER             1910
                   ...  
ARMSTRONG             17
DONNELLSON            17
BUSSEY                16
GILMORE CITY          16
DANVILLE              15
GOLDFIELD             15
WASHBURN              15
SCHALLER              15
VAN METER             15


In [27]:
iowa.zip_code.value_counts()

50010    7077
52402    6938
52240    6128
50613    5267
52001    4755
51501    4652
50314    4519
50317    4425
50265    4356
52404    4242
50401    4119
52722    3699
52807    3530
52405    3502
52241    3446
52761    3389
50311    3384
51503    3382
50320    3237
52501    3206
50702    3175
50315    3091
52804    2973
50501    2972
52601    2952
50703    2885
50322    2880
50266    2843
52732    2822
50158    2682
         ... 
52625      17
50514      17
50044      16
50541      16
50542      15
50261      15
52623      15
51053      15
51002      14
51466      14
50071      13
50251      12
50830      12
52223      11
50150      11
51005      11
51038      10
51553       9
52337       9
50452       8
51338       7
50061       6
50540       6
50162       6
50634       6
51535       5
51530       3
51453       3
52801       2
52328       2
Name: zip_code, dtype: int64

In [26]:
iowa.county_number.value_counts()

77.0    48944
57.0    23462
82.0    16630
7.0     15030
52.0    13163
78.0     9088
85.0     8944
97.0     8541
31.0     7739
17.0     6360
29.0     4082
70.0     3975
23.0     3569
90.0     3522
30.0     3409
56.0     3319
94.0     3144
64.0     2984
50.0     2828
11.0     2737
25.0     2707
63.0     2601
91.0     2460
9.0      2240
8.0      2105
79.0     2087
21.0     1917
14.0     1911
53.0     1871
71.0     1720
        ...  
37.0      675
99.0      671
83.0      661
47.0      634
45.0      606
46.0      588
1.0       584
38.0      566
76.0      525
65.0      507
58.0      484
59.0      475
19.0      464
39.0      437
13.0      424
12.0      402
98.0      387
41.0      363
68.0      352
72.0      351
54.0      343
87.0      298
89.0      245
2.0       234
5.0       227
27.0      223
26.0      203
80.0      201
93.0      160
36.0       27
Name: county_number, dtype: int64

In [28]:
iowa.county.value_counts()

Polk             48944
Linn             23462
Scott            16630
Black Hawk       15030
Johnson          13163
Pottawattamie     9088
Story             8944
Woodbury          8541
Dubuque           7739
Cerro Gordo       6360
Des Moines        4082
Muscatine         3975
Clinton           3569
Wapello           3522
Dickinson         3409
Lee               3319
Webster           3144
Marshall          2984
Jasper            2828
Buena Vista       2737
Dallas            2707
Marion            2601
Warren            2460
Bremer            2240
Boone             2105
Poweshiek         2087
Clay              1917
Carroll           1911
Jones             1871
O'Brien           1720
                 ...  
Greene             675
Wright             671
Shelby             661
Ida                634
Howard             606
Humboldt           588
Adair              584
Grundy             566
Pocahontas         525
Mills              507
Louisa             484
Lucas              475
Chickasaw  

In [30]:
#hmm is this column even really useful? category_name is much more informative...
iowa.category.value_counts()

1031080.0    35256
1012100.0    26967
1011200.0    15290
1062310.0    14575
1031200.0    13931
1022100.0    12044
1011100.0    11507
1081600.0    10865
1032080.0    10640
1062200.0    10020
1062300.0     7245
1011300.0     7058
1071100.0     6883
1051010.0     6560
1041100.0     6542
1032200.0     6471
1082900.0     6281
1081200.0     6256
1012200.0     5358
1052010.0     4592
1081390.0     4229
1081900.0     3387
1031100.0     3300
1081300.0     3197
1042100.0     2923
1012300.0     2781
1012210.0     2285
1081030.0     1908
1081330.0     1643
1051110.0     1578
             ...  
1081370.0      479
1062050.0      393
1062100.0      385
1081317.0      333
1051120.0      330
1081340.0      324
1011250.0      318
1081335.0      304
1081355.0      301
1081015.0      273
1081240.0      256
1041150.0      249
1081365.0      234
1081230.0      200
1011400.0      190
1041200.0      168
1081220.0      157
1051150.0      142
1081350.0       96
1081500.0       86
1031090.0       77
1081250.0   

In [29]:
#this could definitely be simplified into fewer categories
iowa.category_name.value_counts()

VODKA 80 PROOF                        35256
CANADIAN WHISKIES                     26967
STRAIGHT BOURBON WHISKIES             15290
SPICED RUM                            14575
VODKA FLAVORED                        13931
TEQUILA                               12044
BLENDED WHISKIES                      11507
WHISKEY LIQUEUR                       10865
IMPORTED VODKA                        10640
PUERTO RICO & VIRGIN ISLANDS RUM      10020
FLAVORED RUM                           7245
TENNESSEE WHISKIES                     7058
AMERICAN COCKTAILS                     6883
AMERICAN GRAPE BRANDIES                6560
AMERICAN DRY GINS                      6542
IMPORTED VODKA - MISC                  6471
MISC. IMPORTED CORDIALS & LIQUEURS     6281
CREAM LIQUEURS                         6256
SCOTCH WHISKIES                        5358
IMPORTED GRAPE BRANDIES                4592
IMPORTED SCHNAPPS                      4229
MISC. AMERICAN CORDIALS & LIQUEURS     3387
100 PROOF VODKA                 

In [31]:
iowa.vendor_number.value_counts()

260    46605
434    26999
65     25458
370    16861
115    15792
421    13649
55     12745
35     12436
85     12294
259    10412
380    10375
395     9225
205     8547
297     8517
300     5888
305     3490
420     3442
461     3325
192     2742
322     2609
240     2344
410     1954
389     1931
301     1848
255     1669
330     1225
125      817
277      614
384      524
492      523
       ...  
413        8
495        8
399        7
226        6
250        6
210        6
503        5
269        5
101        5
206        4
293        3
198        3
376        3
268        2
452        2
377        2
166        2
224        2
446        2
80         2
215        2
448        1
161        1
90         1
187        1
477        1
310        1
432        1
459        1
363        1
Name: vendor_number, dtype: int64

In [32]:
iowa.item_number.value_counts()

36308     2789
11788     2736
35918     1928
43336     1661
11776     1647
64866     1561
64858     1539
36306     1441
36904     1364
43028     1282
37996     1260
25608     1257
43337     1250
11296     1249
11774     1197
36908     1142
26826     1140
34006     1138
45278     1134
37348     1087
43338     1083
10550     1077
11777     1050
43334     1005
35318      988
12408      978
42716      970
38176      933
36969      927
11297      914
          ... 
994763       1
63536        1
903717       1
1071         1
900248       1
941941       1
932343       1
904841       1
985552       1
561          1
904227       1
36896        1
4656         1
900485       1
41506        1
964068       1
985298       1
902027       1
36232        1
904480       1
900228       1
902795       1
489          1
903713       1
903308       1
900482       1
935677       1
34183        1
943451       1
64138        1
Name: item_number, dtype: int64

In [33]:
iowa.item_description.value_counts()

Black Velvet                              7208
Hawkeye Vodka                             6673
Five O'clock Vodka                        4493
Smirnoff Vodka 80 Prf                     3426
Captain Morgan Spiced Rum                 3338
Seagrams 7 Crown Bl Whiskey               3270
Fireball Cinnamon Whiskey                 3187
Jack Daniels Old #7 Black Lbl             3096
Absolut Swedish Vodka 80 Prf              2910
Admiral Nelson Spiced Rum                 2802
Bacardi Superior Rum                      2802
Jim Beam                                  2717
Mccormick Vodka Pet                       2506
Jagermeister Liqueur                      2408
Five Star                                 2287
Crown Royal Canadian Whisky               2285
Grey Goose Vodka                          2264
Paramount White Rum                       2255
Jose Cuervo Especial Reposado Tequila     2200
Southern Comfort                          2088
Uv Blue (raspberry) Vodka                 2064
Malibu Coconu

In [34]:
iowa.bottle_volume_ml.value_counts()

750     121989
1750     54097
1000     36637
375      26863
500      11942
200       9852
600       3274
3000      1600
300       1334
100        855
800        317
1200       250
2400       103
400         29
850         24
4800        17
50          14
3600        13
603         10
6000         7
950          6
2550         6
2250         6
1125         5
150          3
900          2
4500         1
1500         1
250          1
Name: bottle_volume_ml, dtype: int64

In [35]:
#lots of ones...
iowa.state_bottle_cost.value_counts()

8.25      6110
6.50      4742
9.00      4370
10.00     4213
10.49     3997
15.00     3805
7.17      2991
7.47      2944
5.00      2796
6.30      2777
7.00      2750
5.23      2618
11.49     2500
6.90      2473
7.49      2342
3.34      2334
7.62      2282
6.92      2227
8.20      2162
8.98      2155
3.37      2122
12.50     2098
11.00     2063
18.49     2051
4.75      2024
7.20      1928
3.50      1911
5.50      1868
18.00     1846
10.50     1738
          ... 
4.60         1
35.38        1
35.60        1
45.48        1
31.31        1
1.32         1
14.60        1
28.07        1
29.50        1
24.53        1
6.87         1
28.50        1
15.92        1
9.43         1
15.72        1
5.07         1
7.73         1
20.67        1
11.35        1
16.54        1
6.37         1
9.63         1
17.06        1
299.99       1
16.47        1
5.17         1
5.83         1
10.73        1
52.47        1
30.42        1
Name: state_bottle_cost, dtype: int64

In [36]:
iowa.state_bottle_retail.value_counts()

12.38     6110
9.75      4783
13.50     4415
15.00     4222
22.50     3814
15.74     3809
10.76     2991
11.21     2944
7.50      2796
9.45      2777
10.50     2761
7.85      2618
17.24     2498
10.35     2473
11.24     2342
5.01      2334
11.43     2282
10.38     2227
12.30     2162
13.47     2142
5.06      2122
18.75     2116
16.50     2063
27.74     2051
7.13      2024
10.80     1928
5.25      1911
8.25      1869
27.00     1855
15.75     1740
          ... 
21.90        1
180.33       1
54.39        1
51.54        1
4.41         1
86.31        1
45.77        1
78.70        1
20.84        1
17.22        1
29.98        1
67.50        1
30.89        1
14.30        1
89.96        1
7.76         1
24.71        1
24.81        1
5.87         1
8.47         1
57.89        1
31.88        1
20.01        1
1.98         1
68.22        1
56.18        1
9.18         1
4.64         1
24.99        1
93.03        1
Name: state_bottle_retail, dtype: int64

In [37]:
iowa.bottles_sold.value_counts()

12      72607
6       51887
2       37321
1       31058
3       27758
4       15051
24      14798
48       3765
5        3189
36       2115
18       1734
10       1383
8        1227
60       1098
30        591
72        488
7         397
120       379
96        302
84        200
9         168
144       151
15        117
180       108
42        107
240       107
300       100
90         89
150        86
44         66
        ...  
1116        1
157         1
372         1
2400        1
88          1
354         1
594         1
81          1
615         1
504         1
588         1
75          1
840         1
1128        1
624         1
57          1
282         1
1080        1
378         1
564         1
816         1
390         1
396         1
39          1
37          1
97          1
402         1
28          1
1050        1
33          1
Name: bottles_sold, dtype: int64

In [38]:
iowa.sale_dollars.value_counts()

162.00      3468
148.56      2536
64.80       2066
94.20       2006
70.56       1889
90.00       1772
60.12       1711
73.80       1626
62.28       1624
117.00      1594
60.72       1556
188.88      1556
64.56       1548
180.00      1516
135.00      1511
45.00       1446
126.00      1433
30.00       1363
270.00      1311
161.64      1281
99.00       1260
72.00       1246
81.00       1228
124.20      1182
132.78      1052
58.50       1051
22.50       1004
24.76        992
67.26        982
40.50        979
            ... 
2720.16        1
1474.20        1
8673.96        1
104.52         1
67.49          1
3301.44        1
374.95         1
1817.64        1
191.22         1
44.08          1
10255.44       1
132.27         1
493.20         1
800.28         1
3862.08        1
850.32         1
276.38         1
59.68          1
977.04         1
26054.40       1
361.80         1
389.52         1
1984.92        1
556.92         1
4253.40        1
5920.20        1
493.92         1
584.64        

In [39]:
iowa.volume_sold_liters.value_counts()

9.00       53048
10.50      35789
1.50       24640
2.25       19608
0.75       17221
12.00      17168
3.00       13619
4.50       11958
1.00        5976
0.50        5660
21.00       5277
2.00        4871
3.50        4000
1.12        3745
6.00        3568
18.00       3567
0.60        3003
24.00       2932
1.75        2928
4.80        2142
1.20        2078
9.60        2065
5.25        1929
0.38        1922
4.00        1858
3.75        1635
31.50       1535
36.00       1389
1.88         986
0.80         971
           ...  
1.10           1
2508.00        1
6.60           1
444.00         1
67.20          1
432.00         1
306.00         1
661.50         1
399.00         1
141.75         1
72.75          1
516.00         1
1837.50        1
2100.00        1
68.25          1
372.00         1
117.75         1
1039.50        1
4.75           1
829.50         1
150.00         1
766.50         1
567.00         1
2.20           1
546.00         1
4.20           1
196.80         1
531.00        

In [40]:
iowa.volume_sold_gallons.value_counts()

2.38      53048
2.77      35789
0.40      24640
0.59      19608
0.20      17221
3.17      17168
0.79      13619
1.19      11958
0.26       5976
0.13       5660
5.55       5277
0.53       4871
0.92       4000
0.30       3745
1.59       3569
4.76       3567
0.16       3003
6.34       2932
0.46       2928
1.27       2142
0.32       2079
2.54       2065
1.39       1929
0.10       1922
1.06       1858
0.99       1635
8.32       1535
9.51       1389
0.50        989
0.21        971
          ...  
98.27         1
1.11          1
499.29        1
3.70          1
194.17        1
0.29          1
1.74          1
0.58          1
475.51        1
17.75         1
105.40        1
2.58          1
144.24        1
0.67          1
117.29        1
28.29         1
162.47        1
4.65          1
149.79        1
183.07        1
19.22         1
5.09          1
228.24        1
32.97         1
31.11         1
215.56        1
80.84         1
4.44          1
20.29         1
485.42        1
Name: volume_sold_gallon

In [22]:
iowa.columns

Index([u'date', u'store_number', u'city', u'zip_code', u'county_number',
       u'county', u'category', u'category_name', u'vendor_number',
       u'item_number', u'item_description', u'bottle_volume_ml',
       u'state_bottle_cost', u'state_bottle_retail', u'bottles_sold',
       u'sale_dollars', u'volume_sold_liters', u'volume_sold_gallons'],
      dtype='object')

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Filter/adjust the store data

---

Some stores may have opened or closed in 2015. These stores will have incorrect yearly summary statistics since they were not open the full year. We need to filter them out or find another way to deal with the inconsistent numbers of months across stores.

It is up to you how you want to deal with this problem.

1. Investigate problematic stores.
2. Decide on an approach to deal with stores not open for the full 2015 year.
3. Implement your plan.
4. Briefly report on what you did and why.


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Proposals for new tax rates (Scenario 1)

---

You are a data scientist in residence at the Iowa State tax board. The Iowa State legislature is considering changes in the liquor tax rates and has assigned you to the project.

### 5.1 Calculate yearly liquor sales for each store in 2015.

### 5.2 Calculate the profit each store is making in 2015.


### 5.3 Investigate which Iowa counties are making the most profit on liquor per gallon in 2015.

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.4 Plot a color map of average profit per county in 2015.

[Check out this site](http://flowingdata.com/2009/11/12/how-to-make-a-us-county-thematic-map-using-free-tools/) for  information on how to create map visualizations in python. There are other resources online for map plotting in python as well.

Another interesting map plot might show the difference in profit between time periods. Change in profit from November to December, for example.

### 5.5  Does profit scale proportionally to store size?

Size of a store can be defined as the total gallons sold in 2015. Is the profit to store size ratio increasing as store size increases? Decreasing? Consistent?

### 5.6 Create a broader category for liquor type.

Liquor types are pretty granular in this dataset. Create a column that categorizes these types into a smaller amount of categories. The categories you decide on are up to you.

### 5.7 Do relationships exist between the broader liquor type and profit?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.8 Are there any noticeable relationships between liquor brand and sales? Profit?

### 5.9 The tax board wants to design a tax or taxes that affect larger stores more than smaller "mom and pop" stores.

Based on your investigations into the data, come up with a way you could design a tax that achieves this goal **without explicitly taxing stores based on size or county critera.** The liqour board does not want to obviously punish larger stores or speific counties for fear of backlash, but is willing to tax hard alcohol more than beer, for example.

Feel free to do more EDA if it helps.

Your report should describe whether such a tax is possible or not, and the specifics of what the tax will target/do.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. Market research for new store locations (Scenario 2)

A liquor store owner in Iowa is looking to open a store in a new location and has hired you to investigate the market data for potential new locations. The business owner is interested in what you can discover from the data that will help him evaluate a good location for a new storefront.

---

### 6.1 Create columns for Q1 2015 sales and Q1 2016 sales.

The owner is not just interested in what locations are currently selling well, but also which areas are showing the most promising sales growth from 2015 to 2016. 

For yearly changes we will focus on the changes from Q1 2015 to Q1 2016 specifically (avoiding effects of season changes).

Note: Q1 is months: January, February, and March.


### 6.2 Decide on a metric for performance that combines Q1 2016 sales as well as growth in sales from Q1 2015.

Again, the owner wants to know that his store will be in a good position to sell a large amount of liquor now, but also have his store be more likely to grow and sell even more going forward. 

In particular, he is worried about opening a store in an area that is showing a decline in liquor sales between Q1 2015 and Q1 2016.

### 6.3 Recommend a location based on your metric.

The owner asks you to make a recommendation for a new store location. Explain why you recommended this area.

### 6.4 Recommend a location based also on a restricted inventory type.

Your employer has now decided he wants his store to focus on one type of liquor only. He askes you:

1. What would be the best type of liquor to sell to maximize profit?
2. Should the recommended location change if he is only selling one type of liquor?

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7. Time-related effects

---

You could imagine that liquor sales might be affected by a variety of effects related to time. Do people buy more beer in the summer? Do liquor sales skyrocket in december? Do people buy less liquor on Tuesdays?

You have the date of sales in your dataset, which you can use to pull out time components.

1. Come up with 3 different hypotheses about how liquor sales may vary with time-related variables. 
2. Create a visualization exploring each hypothesis.
3. Write brief concluding remarks on what you observed.


<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 8. Appropriate spending

---

The owner you worked with before realized that opening new stores cost money, something he had completely overlooked earlier. He brings you back in to help him figure out how much he should be willing to spend on opening his store.

### 8.1 Calculate costs and time to recoup.

The owner has decided he will buy 20,000 gallons of the liquor type you recommended (randomly sampled across the different brands/subtypes). He will also be opening his store in the area you recommended.

The cost of buying the property for his store will be $150,000.

1. How much should he expect to spend on the 20,000 gallons of liquor alone?
2. Based on the amount he's spent on the liquor and property, and on the profit per time period he is expected to have, how long will it take him to recoup the costs?

<img src="http://imgur.com/gGrau8s.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">


### 8.2 Calculate maximum loan interest rate.

The owner unfortunately has to take out a loan to to pay for the entirety of the property and liquor. 

**Conditions of the loan**:

Every quarter, (except for Q1 when he takes out the loan), the interest rate is applied to the remaining balance on the loan and added to his debt. 

If, for example, he made no payments whatsoever, not only would the amount of debt he owed keep increasing by quarter but the debt incurred from interest would keep increasing as well. (Since his debt keeps growing, the interest rate is applied each month to the bigger balance, incurring more debt each time.)

**Determining maximum acceptable interest:**

Given this, _the owner wants to be 99% confident that he will be able to pay off his loans._ If the interest rate is too high he will have to eventually declare bankruptcy.

Based on his expected profits per quarter (assume the expected profit stays the same for all future quarters), what is the maximum interest rate he should accept on the loan to be 99% confident he will be able to pay it off?